# ***Libraries & Constants***

In [ ]:
import os
import pandas as pd
import seaborn as sns
import numpy as np

from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split

from matplotlib import pyplot as plt

In [ ]:
!gdown --id 1-Zyp-JP3f9QhPKaErBkPPFNKaPS1v74u

In [ ]:
categorical_attr = ['gender', 'NationalITy', 'PlaceofBirth', 'StageID', 'GradeID', 'SectionID', 'Topic', 'Semester', 'Relation', 'ParentAnsweringSurvey', 'ParentschoolSatisfaction', 'StudentAbsenceDays']

# ***Preprocessing***

Reading & displaying data:

In [ ]:
df = pd.read_csv('/content/Dataset.csv')
df.head()

In [ ]:
df.loc[(df["Class"] == "M") | (df["Class"] == "H"), "Class"] = 1
df.loc[df["Class"] == "L", "Class"] = 0
print(df.shape)

In [ ]:
# X: Features, y: Classes
y = np.array(df['Class'])
X = np.array(pd.get_dummies(df.iloc[:, :-1])).astype("float")

In [ ]:
# Deviding Dataset to training and validation set
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=12)
normalizer = StandardScaler()
X_train = normalizer.fit_transform(X_train)
X_val = normalizer.transform(X_val)

In [ ]:
print('Number of dataset: ', len(X))
print('Number of train set: ', len(X_train))
print('Number of validation set: ', len(X_val))

# ***Implementing Model***

In [ ]:
class FCLayer:
  def __init__(self, input_size, output_size, landa = 0.00, random=True):
    self.landa = landa
    if random:
      # Xavier Glorot Initialization
      limit = np.sqrt(2 / float(input_size + output_size))
      self.weights = np.random.normal(0.0, limit, size=(input_size, output_size))
      self.bias = np.random.normal(0.0, limit, size=(1, output_size))
    else:
      self.weights = np.zeros((input_size, output_size)) + 1e-15
      self.bias = np.zeros((1, output_size)) + 1e-15

  def forward(self, input):
    self.input = input
    return np.dot(input, self.weights) + self.bias
    
  def backward(self, output_error, learning_rate):
    input_error = np.dot(output_error, self.weights.T)
    weights_error = np.dot(self.input.T, output_error) + (self.landa * (self.weights))

    self.weights -= learning_rate * weights_error
    self.bias -= learning_rate * output_error
    return input_error

In [ ]:
class SigmoidLayer:
  def dsigmoid(self, input):
    return np.exp(-self.input) / (1 + np.exp(-self.input))**2

  def forward(self, input):
    self.input = input
    return 1 / (1 + np.exp(-input))
    
  def backward(self, output_error, dummy):
    return output_error * self.dsigmoid(self.input)

In [ ]:
class BinaryCrossEntropy:
  def __call__(self, y_true, y_pred, epsilon=1e-15):
    return -(y_true * (np.log(y_pred + epsilon)) + (1-y_true) * np.log((1 - y_pred) + epsilon))
  
  def backward(self, y_true, y_pred):
    l = -(y_true / y_pred) + ((1-y_true) / (1-y_pred))
    if np.isnan(l):
      return 0
    elif np.isinf(l):
      return 1
    else:
      return l

In [ ]:
class NeuralNetwork:
  def __init__(self, layers):
    self.model = layers

  def compile(self, lossfunc, metrics):
    self.lossfunc = lossfunc
    self.metrics = metrics

  def fit(self, X, Y, EPOCHS, learning_rate, validation_data=None):
    accs = []
    losses = []
    accs_val = []
    losses_val = []
    

    for epoch in range(EPOCHS):
      acc = 0
      loss = 0
      for x, y_true in zip(X, Y):
        # Forward Phase
        x = x.reshape(1, -1)
        output = x 
        for layer in self.model:
          output = layer.forward(output)
        prediction = 1 if output > 0.5 else 0
        
        # Loss Function
        loss += self.lossfunc(y_true, output)
        acc += 1 if prediction == y_true else 0

        # Backward Phase
        output_error = self.lossfunc.backward(y_true, output)
        for layer in reversed(self.model):
            output_error = layer.backward(output_error, learning_rate)
      
      # training data
      acc /= len(X)
      loss /= len(X)
      accs.append(acc)
      losses.append(loss.item())
          

      # validation data
      if validation_data:
        lossval, accval = self.evaluate(validation_data[0], validation_data[1])
        accs_val.append(accval)
        losses_val.append(lossval)
        if epoch % 10 == 0:
          print(f"{epoch+1}/{EPOCHS}, loss={loss.item():.2f}, accuracy={acc:.2f}, validation loss={lossval:.2f}, validation accuracy={accval:.2f}")
      else:
        if epoch % 10 == 0:
          print(f"{epoch+1}/{EPOCHS}, loss={loss.item():.2f}, accuracy={acc:.2f}")

    if validation_data:
      return losses, accs, losses_val, accs_val
    return losses, accs
    
  def evaluate(self, X, Y):
    acc = 0
    loss = 0
    for x, y_true in zip(X, Y):
      # Forward Phase
      x = x.reshape(1, -1)
      output = x 
      for layer in self.model:
        output = layer.forward(output)

      pred = 1 if output > 0.5 else 0
      
      # Loss Function
      loss += self.lossfunc(y_true, output)
      acc += 1 if pred == y_true else 0
    acc /= len(X)
    return loss.item() / len(X), acc

# ***Training the model***

## 2 Layer random initialized

In [ ]:
model = NeuralNetwork([
         FCLayer(X_train.shape[1], 1, random=True),
         SigmoidLayer(),
])

model.compile(BinaryCrossEntropy(), ["accuracy"])

losses, accs, losses_val, accs_val = model.fit(X_train,
          y_train,
          EPOCHS=1000,
          learning_rate=0.01,
          validation_data=(X_val, y_val)
          )

In [ ]:
figure, axis = plt.subplots(2, 2, figsize=(20, 10))

axis[0][0].plot(losses)
axis[0][0].legend(["training loss"])

axis[0][1].plot(accs)
axis[0][1].legend(["training accuracy"])

axis[1][0].plot(losses_val)
axis[1][0].legend(["validation loss"])

axis[1][1].plot(accs_val)
axis[1][1].legend(["validation accuracy"])

lossa, acca =  model.evaluate(X_val, y_val)

print("Model loss on validaiton dataste:", lossa)
print("Model accuracy on validaiton dataste:", acca)
print("Maximum accuracy before overfit happening:", max(accs_val))

plt.savefig('2layer_r.png', bbox_inches='tight')

## 2 layer zero initialized

In [ ]:
model = NeuralNetwork([
         FCLayer(X_train.shape[1], 1, random=False),
         SigmoidLayer(),
])

model.compile(BinaryCrossEntropy(), ["accuracy"])

losses, accs, losses_val, accs_val = model.fit(X_train,
          y_train,
          EPOCHS=100,
          learning_rate=0.01,
          validation_data=(X_val, y_val)
          )

In [ ]:
figure, axis = plt.subplots(2, 2, figsize=(20, 10))

axis[0][0].plot(losses)
axis[0][0].legend(["training loss"])

axis[0][1].plot(accs)
axis[0][1].legend(["training accuracy"])

axis[1][0].plot(losses_val)
axis[1][0].legend(["validation loss"])

axis[1][1].plot(accs_val)
axis[1][1].legend(["validation accuracy"])

lossa, acca =  model.evaluate(X_val, y_val)

print("Model loss on validaiton dataste:", lossa)
print("Model accuracy on validaiton dataste:", acca)
print("Maximum accuracy before overfit happening:", max(accs_val))

plt.savefig('2layer_z.png', bbox_inches='tight')


## 3 Layer random initialized

In [ ]:
model = NeuralNetwork([
         FCLayer(X_train.shape[1], 32, random=True),
         SigmoidLayer(),
         FCLayer(32, 1, random=True),
         SigmoidLayer(),
])

model.compile(BinaryCrossEntropy(), ["accuracy"])

losses, accs, losses_val, accs_val = model.fit(X_train,
          y_train,
          EPOCHS=200,
          learning_rate=0.01,
          validation_data=(X_val, y_val)
          )

In [ ]:
figure, axis = plt.subplots(2, 2, figsize=(20, 10))

axis[0][0].plot(losses)
axis[0][0].legend(["training loss"])

axis[0][1].plot(accs)
axis[0][1].legend(["training accuracy"])

axis[1][0].plot(losses_val)
axis[1][0].legend(["validation loss"])

axis[1][1].plot(accs_val)
axis[1][1].legend(["validation accuracy"])

lossa, acca =  model.evaluate(X_val, y_val)

print("Model loss on validaiton dataste:", lossa)
print("Model accuracy on validaiton dataste:", acca)
print("Maximum accuracy before overfit happening:", max(accs_val))

plt.savefig('3layer_r.png', bbox_inches='tight')

## 3 layer zero initialized

In [ ]:
model = NeuralNetwork([
         FCLayer(X_train.shape[1], 32, random=False),
         SigmoidLayer(),
         FCLayer(32, 1, random=False),
         SigmoidLayer(),
])

model.compile(BinaryCrossEntropy(), ["accuracy"])

losses, accs, losses_val, accs_val = model.fit(X_train,
          y_train,
          EPOCHS=100,
          learning_rate=0.01,
          validation_data=(X_val, y_val)
          )

In [ ]:
figure, axis = plt.subplots(2, 2, figsize=(20, 10))

axis[0][0].plot(losses)
axis[0][0].legend(["training loss"])

axis[0][1].plot(accs)
axis[0][1].legend(["training accuracy"])

axis[1][0].plot(losses_val)
axis[1][0].legend(["validation loss"])

axis[1][1].plot(accs_val)
axis[1][1].legend(["validation accuracy"])

lossa, acca =  model.evaluate(X_val, y_val)

print("Model loss on validaiton dataste:", lossa)
print("Model accuracy on validaiton dataste:", acca)
print("Maximum accuracy before overfit happening:", max(accs_val))

plt.savefig('3layer_z.png', bbox_inches='tight')

## 5 Layer random initialized

In [ ]:
model = NeuralNetwork([
         FCLayer(X_train.shape[1], 32, random=True),
         SigmoidLayer(),
         FCLayer(32, 32, random=True),
         SigmoidLayer(),
         FCLayer(32, 32, random=True),
         SigmoidLayer(),
         FCLayer(32, 1, random=True),
         SigmoidLayer(),
])

model.compile(BinaryCrossEntropy(), ["accuracy"])

losses, accs, losses_val, accs_val = model.fit(X_train,
          y_train,
          EPOCHS=400,
          learning_rate=0.01,
          validation_data=(X_val, y_val)
          )

In [ ]:
figure, axis = plt.subplots(2, 2, figsize=(20, 10))

axis[0][0].plot(losses)
axis[0][0].legend(["training loss"])

axis[0][1].plot(accs)
axis[0][1].legend(["training accuracy"])

axis[1][0].plot(losses_val)
axis[1][0].legend(["validation loss"])

axis[1][1].plot(accs_val)
axis[1][1].legend(["validation accuracy"])

lossa, acca =  model.evaluate(X_val, y_val)

print("Model loss on validaiton dataste:", lossa)
print("Model accuracy on validaiton dataste:", acca)
print("Maximum accuracy before overfit happening:", max(accs_val))

plt.savefig('5layer_r.png', bbox_inches='tight')

## 5 layer zero initialized

In [ ]:
model = NeuralNetwork([
         FCLayer(X_train.shape[1], 32, random=False),
         SigmoidLayer(),
         FCLayer(32, 32, random=False),
         SigmoidLayer(),
         FCLayer(32, 32, random=False),
         SigmoidLayer(),
         FCLayer(32, 1, random=False),
         SigmoidLayer(),
])

model.compile(BinaryCrossEntropy(), ["accuracy"])

losses, accs, losses_val, accs_val = model.fit(X_train,
          y_train,
          EPOCHS=300,
          learning_rate=0.01,
          validation_data=(X_val, y_val)
          )

In [ ]:
figure, axis = plt.subplots(2, 2, figsize=(20, 10))

axis[0][0].plot(losses)
axis[0][0].legend(["training loss"])

axis[0][1].plot(accs)
axis[0][1].legend(["training accuracy"])

axis[1][0].plot(losses_val)
axis[1][0].legend(["validation loss"])

axis[1][1].plot(accs_val)
axis[1][1].legend(["validation accuracy"])

lossa, acca =  model.evaluate(X_val, y_val)

print("Model loss on validaiton dataste:", lossa)
print("Model accuracy on validaiton dataste:", acca)
print("Maximum accuracy before overfit happening:", max(accs_val))

plt.savefig('5layer_z.png', bbox_inches='tight')